##### Import Files

In [14]:
import os
import re
import nibabel as nib
import numpy as np
import torch
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, LazyConv3d , MaxPool3d, Module, Softmax, BatchNorm3d, Dropout, Conv3d
from torch.optim import Adam, SGD

1.11.0+cu113
True


##### DATA LOADING

In [ ]:
!unzip /content/Raw\ Files.zip -d /content/
!unzip /content/Baseline.zip -d /content/

###### Data

In [13]:
def get_mri_data(path_mri):
    '''Returns a list containing the path of MRI DATA'''
    path = path_mri;
    regex = re.compile(r'__MPR.*') ; fileRegex1 = re.compile(r'(WIP)*T13D'); mri_data_path = []; 
    # Iterating over Directories 
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Storing the path of Data
            if regex.search(file) == None:
                if fileRegex1.search(file): mri_data_path.append(os.path.join(subdir, file));
    return mri_data_path;

def get_asl_data(path_asl):
    '''Returns a list containing the path of ASL DATA'''
    asl_path = path_asl;
    asl_regex = re.compile(r'oxasl_dist.*'); asl_data_path = [];
    # Iterating over directories
    for subdir, dirs, files in os.walk(asl_path):
        # Selecting the ASL file
        for file in files:
            if file == 'perfusion_calib.img':
                if asl_regex.search(subdir): 
                    asl_data_path.append(os.path.join(subdir, file));
    return asl_data_path;

def create_data_dict(mri_data_path,asl_data_path): 
    '''To create a Dictionary of MRI_data and ASL_data'''
    mri_map = {}; asl_map = {};
    ## Iterating over the mri data's path
    for i in mri_data_path:
    ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/')[4].split(' ')[0]; mri_map[key] = value;
    ## Iterating over the mri data's path
    for i in asl_data_path:
        ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/')[3].split(' ')[0];asl_map[key] = value;
    return mri_map,asl_map;

def create_tensors(mri_data_dict,asl_data_dict,device):
    '''Creates tensors from the data dictionary feeded'''
    x = [] ; y = []; X = []; Y = [];
    for k,v in mri_data_dict.items():
        if k in asl_data_dict:
            # Loading the MRI image from the path in the train x path 
            mri_img = nib.load(v); 
            # Making it a numpy array
            mri_vec = np.array(mri_img.dataobj)[np.newaxis,:,:,:] # Channels x Length X Breadth X Slices of Brain
            # Min max Normalizing the image 
            mri_vec = (mri_vec - mri_vec.min()) / (mri_vec.max() - mri_vec.min())
            # Appending the MRI image to X 
            x.append(torch.as_tensor(mri_vec,dtype=torch.float32).to(device)); 
            ## Finding the same patient with ASL data 
            asl_img = nib.load(asl_data_dict[k]); asl_vec = np.array(asl_img.dataobj)[np.newaxis,:,:,:]; 
            asl_vec = (asl_vec - asl_vec.min()) / (asl_vec.max() - asl_vec.min());
            ## Appending the image to y
            y.append(torch.as_tensor(asl_vec,dtype=torch.float32).to(device))
            X = torch.stack(x,dim=0).to(device); Y = torch.stack(y,dim=0).to(device);
    return X,Y

def print_data_shape(data):
    '''Prints the shape of data'''
    for i in data: print(i.shape);

def tensor_stats(tensor_array):
    '''Prints some statistics about the tensors'''
    for i in tensor_array:
        print(i.sum(), i.prod(), i.mean(), i.std());

def print_data_dimension(data):
    '''Print dimension of data'''
    print(data.shape)

def data_loader(path_mri,path_asl,device):
    '''This function will return us the mri and asl data in format we want'''
    ## Getting path of all MRI data stored and ASL data stored
    mri_data = get_mri_data(path_mri); asl_data = get_asl_data(path_asl);
    ## Making dict of MRI data for which ASL data exists
    mri_data_dict , asl_data_dict = create_data_dict(mri_data,asl_data);
    ## Creating the Tensors of the MRI and ASL data
    x , y = create_tensors(mri_data_dict,asl_data_dict,device);
    return x,y

## Change Path accordingly
path_mri = '/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/' ; path_asl = '/content/Baseline/';

## Getting the data
x,y = data_loader(path_mri,path_asl,device)

## Printing the shape of specifid data
print('Shape of MRI images - >');       print_data_shape(x); 
print('\nShape of ASL images - >');     print_data_shape(y);

## Printing the Stats of the data
print('\nStats for MRI data(X) - >');   tensor_stats(x); 
print('\nStats for ASL data(X) - >');   tensor_stats(y);

## Printing the Dimension of X and Y  
## Dimension refer -> Number x Length x Breadth x Height x Channel
print('\nDimensions of X(MRI Data) ->',end=' ');print_data_dimension(x)
print('\nDimensions of Y(ASL Data) ->',end=' ');print_data_dimension(y) 

Shape of MRI images - >
torch.Size([1, 288, 288, 180])
torch.Size([1, 288, 288, 180])
torch.Size([1, 288, 288, 180])
torch.Size([1, 288, 288, 180])

Shape of ASL images - >
torch.Size([1, 80, 80, 13])
torch.Size([1, 80, 80, 13])
torch.Size([1, 80, 80, 13])
torch.Size([1, 80, 80, 13])

Stats for MRI data(X) - >
tensor(1135785.3750, device='cuda:0') tensor(0., device='cuda:0') tensor(0.0761, device='cuda:0') tensor(0.1223, device='cuda:0')
tensor(882671.6250, device='cuda:0') tensor(0., device='cuda:0') tensor(0.0591, device='cuda:0') tensor(0.1196, device='cuda:0')
tensor(1256232.7500, device='cuda:0') tensor(0., device='cuda:0') tensor(0.0841, device='cuda:0') tensor(0.1339, device='cuda:0')
tensor(632351.7500, device='cuda:0') tensor(0., device='cuda:0') tensor(0.0424, device='cuda:0') tensor(0.0786, device='cuda:0')

Stats for ASL data(X) - >
tensor(4592.5898, device='cuda:0') tensor(0., device='cuda:0') tensor(0.0552, device='cuda:0') tensor(0.1219, device='cuda:0')
tensor(6011.9717

#### Model

In [15]:
hidden = lambda c_in, c_out: Sequential(
    Conv3d(c_in, c_out, (32,32,26)),
    BatchNorm3d(c_out),
    ReLU(),
    MaxPool3d(2)
    )
class Model_1(Module):
    '''Class for the Model to be fitted on MRI data'''
    def __init__(self, c):
        super(Model_1, self).__init__();
        self.hidden1 = hidden(1,c);
        self.hidden2 = hidden(c,2*c);
        self.hidden3 = hidden(2*c,4*c);
        self.hidden4 = hidden(4*c, 8*c);
        self.hidden5 = hidden(8*c, 4*c);
        self.hidden6 = hidden(4*c,c);
        self.hidden7 = hidden(c,1);
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.hidden4(x)
        x = self.hidden5(x)
        x = self.hidden6(x)
        x = self.hidden7(x)
        return x 


In [26]:
input  = 180 ; output = 13 ; kernel = 26 ; ctr = 0;

while(input >= output):
    input = input - kernel; ctr = ctr+1;

print(ctr)

kernel = 32 , 26(height) 

7


### WASTE

###### LOSS

In [10]:
def mse(y_t , y_p):
    ## To calculate the mean square error
    print(y_t.shape); print(y_p.shape); 
    ## Normal Mean Square Error - reduce( x+y , (y_t - y_p)^2)
    loss = torch.sum(torch.square(torch.flatten(y_t) - torch.flatten(y_p)))/len(torch.flatten(y_t))
    return loss
print(mse(y[1],y[0]))

torch.Size([80, 80, 13, 1])
torch.Size([80, 80, 13, 1])
tensor(0.0121, device='cuda:0')


##### Training



In [93]:
## Go with Lazy 3D convs LAZYCONV3Dw

## The loss function to get the result
class Normal_3D_cnn(Module):   
    
    def __init__(self):
        super(Normal_3D_cnn, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 3D convolution layer
            LazyConv3d(1, kernel_size= 3, stride=1, padding=1),
            BatchNorm3d(1),
            ReLU(inplace=True),
            MaxPool3d(kernel_size=2, stride=2),
            # # Defining another 3D convolution layer
            # LazyConv3d(1,kernel_size=3, stride=1, padding=1),
            # BatchNorm3d(1),
            # ReLU(inplace=True),
            # MaxPool3d(kernel_size=2, stride=2),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        return x

model = Normal_3D_cnn()
print(model)
optimizer = Adam(model.parameters(), lr=0.07)

def train():
    model.train()
    optimizer.zero_grad()
    y_p = model(x[0])
    loss = mse(y[0],y_p)
    return loss, y_p

loss,y_p = train()
print(loss,y_p.shape)
